In [42]:
import numpy as np
import torch
import torch.nn as nn

d_k = 64 # K（=Q）维度
d_v = 64 # V维度
# 定义缩放点积注意力类
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()
    def forward(self, Q, K, V, attn_mask):
        #----------------------维度信息------------------------
        # Q、K、V [batch_size, n_heads, len_q/k/v, dim_k/v] (dim_q=dim_k)
        # attn_mask [batch_size, n_heads, len_q, len_k]
        #-------------------------------------------------------
        # 计算注意力分数（原始权重）[batch_size, n_heads, len_q, dim_q] * [batch_size, n_heads, dim_k, len_q] = [batch_size, n_heads, len_q, len_k]
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)
        #----------------------维度信息------------------------
        # scores [batch_size, n_heads, len_q, len_k]
        #-------------------------------------------------------
        # 使用注意力掩码，将attn_mask中值为1的位置的权重替换为极小值
        #----------------------维度信息------------------------
        # attn_mask [batch_size, n_heads, len_q, len_k]
        #-------------------------------------------------------
        scores.masked_fill_(attn_mask, -1e9)
        # 用softmax函数对scores进行归一化，得到注意力权重
        weights = nn.Softmax(dim=-1)(scores)  # 创建了一个指定最后一个维度的softmax层，并对scores进行softmax操作
        #----------------------维度信息------------------------
        # weights [batch_size, n_heads, len_q, len_k]
        #-------------------------------------------------------
        # 计算上下文向量（也就是注意力值），是上下文信息的紧凑表示 [batch_size, n_heads, len_q, len_k] * [batch_size, n_heads, len_k, dim_v] = [batch_size, n_heads, len_q, dim_v]
        context = torch.matmul(weights, V)
        #----------------------维度信息------------------------
        # context [batch_size, n_heads, len_q, dim_v]
        #-------------------------------------------------------
        return context, weights # 返回上下文信息和注意力权重

In [43]:
# 定义多头注意力类
d_embedding = 512 # 词嵌入维度
n_heads = 8 # 多头注意力个数
batch_size = 3 # 批次大小
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        # 确保d_model能被n_heads整除
        assert d_embedding % n_heads == 0
        self.W_Q = nn.Linear(d_embedding, d_k * n_heads)  # Q的线性变换层
        self.W_K = nn.Linear(d_embedding, d_k * n_heads)  # K的线性变换层
        self.W_V = nn.Linear(d_embedding, d_v * n_heads)  # V的线性变换层
        self.linear = nn.Linear(d_v * n_heads, d_embedding)  # 最后的线性变换层
        self.layer_norm = nn.LayerNorm(d_embedding)  # Layer Norm层
    def forward(self, Q, K, V, attn_mask):
        #----------------------维度信息------------------------
        # Q、K、V [batch_size, len_q/k/v, embedding_dim]
        #-------------------------------------------------------
        residual, batch_size = Q, Q.size(0) # 残差连接
        # 将输入进行线性变换和重塑，以便后续处理，[batch_size, len_q/k/v, embedding_dim] -> [batch_size, n_heads, len_q/k/v, d_k/d_v]
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1, 2)  # Q [batch_size, n_heads, len_q, d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1, 2)  # K [batch_size, n_heads, len_k, d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1, 2)  # V [batch_size, n_heads, len_v, d_v]
        # 将注意力掩码复制多到头 attn_mask [batch_size, 1, len_q, len_k] -> [batch_size, n_heads, len_q, len_k]
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)
        #----------------------维度信息------------------------
        # attn_mask [batch_size, n_heads, len_q, len_k]
        #-------------------------------------------------------
        # 使用缩放点积注意力计算上下文和注意力权重
        context, weights = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        #----------------------维度信息------------------------
        # context [batch_size, n_heads, len_q, d_v]
        # weights [batch_size, n_heads, len_q, len_k]
        #-------------------------------------------------------
        # 通过调整维度将多个头的上下文向量连接在一起，[batch_size, n_heads, len_q, d_v] -> [batch_size, len_q, n_heads * d_v]
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v)
        #----------------------维度信息------------------------
        # context [batch_size, len_q, n_heads * d_v]
        #-------------------------------------------------------
        # 用一个线性层把连接后的在多头注意力结果转换，原始地嵌入维度，[batch_size, len_q, n_heads * d_v] -> [batch_size, len_q, embedding_dim]
        output = self.linear(context)
        #----------------------维度信息------------------------
        # output [batch_size, len_q, embedding_dim]
        #-------------------------------------------------------
        # 与输入(Q)进行残差连接，并进行层归一化后处理
        output = self.layer_norm(output + residual)
        #----------------------维度信息------------------------
        # output [batch_size, len_q, embedding_dim]
        #-------------------------------------------------------
        return output, weights  # 返回层归一化的输出和和注意力权重

In [44]:
# 定义逐位置前馈网络
class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, d_ff=2048):
        super(PoswiseFeedForwardNet, self).__init__()
        # 定义一维卷积层1，用于将输入映射到更高维度
        self.conv1 = nn.Conv1d(in_channels=d_embedding, out_channels=d_ff, kernel_size=1)
        # 定义一维卷积层2，用于将映射后的向量映射回原始维度
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_embedding, kernel_size=1)
        # 定义层归一化
        self.layer_norm = nn.LayerNorm(d_embedding)
    def forward(self, inputs):
        #----------------------维度信息------------------------
        # inputs [batch_size, len_q, embedding_dim]
        #-------------------------------------------------------
        residual = inputs  # 保留残差连接
        # 在第一个卷积层1后使用ReLU函数，[batch_size, len_q, embedding_dim] -> [batch_size, embedding_dim, len_q] -> [batch_size, len_q, d_ff]
        output = nn.ReLU()(self.conv1(inputs.transpose(1, 2)))
        #----------------------维度信息------------------------
        # output [batch_size, d_ff, len_q]
        #-------------------------------------------------------
        # 使用卷积2进行降维
        output = self.conv2(output).transpose(1, 2)
        #----------------------维度信息------------------------
        # output [batch_size, len_q, embedding_dim]
        #-------------------------------------------------------
        # 与输入进行残差连接，并进行层归一化
        output = self.layer_norm(output + residual)
        #----------------------维度信息------------------------
        # output [batch_size, len_q, embedding_dim]
        #-------------------------------------------------------
        return output  # 返回层归一化的输出

In [45]:
# 生成正弦位置编码表的函数，用于在Transformer中引入位置信息
def get_sin_enc_table(n_position, embedding_dim):
    #----------------------维度信息------------------------
    # n_position: 输入序列的最大长度
    # embedding_dim: 词嵌入的维度
    #-------------------------------------------------------
    # 根据位置和维度信息，初始化正弦位置编码表
    sinusoid_table = np.zeros((n_position, embedding_dim))
    # 遍历所有的位置和维度，计算角度值
    for pos_i in range(n_position):
        for hid_j in range(embedding_dim):
            # 根据位置和维度计算角度值
            angle = pos_i / np.power(10000, 2 * (hid_j // 2) / embedding_dim)
            sinusoid_table[pos_i, hid_j] = angle
    # 计算正弦和余弦值
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # dim 2i偶数堆
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # dim 2i+1奇数堆
    #----------------------维度信息------------------------
    # sinusoid_table [n_position, embedding_dim]
    #-------------------------------------------------------
    return torch.FloatTensor(sinusoid_table)  # 返回正弦位置编码表

In [46]:
# 定义填充注意力掩码函数
def get_attn_pad_mask(seq_q, seq_k):
    #----------------------维度信息------------------------
    # seq_q的维度是[batch_size, len_q]
    # seq_k的维度是[batch_size, len_k]
    #-------------------------------------------------------
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # 生成布尔类型张量
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # <PAD>token的编码值为0
    #----------------------维度信息------------------------
    # pad_attn_mask [batch_size, 1, len_k]
    #-------------------------------------------------------
    # 变形为与注意力分数相同的张量
    pad_attn_mask = pad_attn_mask.expand(batch_size, len_q, len_k)
    #----------------------维度信息------------------------
    # pad_attn_mask [batch_size, len_q, len_k]
    #-------------------------------------------------------
    return pad_attn_mask  # 返回注意力掩码张量

In [47]:
# 定义编码器层类
class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()  # 多头注意力层
        self.pos_ffn = PoswiseFeedForwardNet()  # 前馈神经网络层
    def forward(self, enc_inputs, enc_self_attn_mask):
        #----------------------维度信息------------------------
        # enc_inputs [batch_size, seq_len, embedding_dim]
        # enc_self_attn_mask [batch_size, seq_len, seq_len]
        #-------------------------------------------------------
        # 将相同的Q，K，V输入到多头注意力层，返回的attn_weights增加了头数
        enc_outputs, attn_weights = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask)
        #----------------------维度信息------------------------
        # enc_outputs [batch_size, seq_len, embedding_dim]
        # attn_weights [batch_size, n_heads, seq_len, seq_len]
        #-------------------------------------------------------
        # 将多头注意力的输出输入到前馈神经网络层
        enc_outputs = self.pos_ffn(enc_outputs)  # 维度与enc_inputs相同
        #----------------------维度信息------------------------
        # enc_outputs [batch_size, seq_len, embedding_dim]
        #-------------------------------------------------------
        return enc_outputs, attn_weights  # 返回前馈神经网络层的输出和注意力权重

In [48]:
# 定义编码器类
n_layers = 6 # 编码器层数
class Encoder(nn.Module):
    def __init__(self, corpus):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(len(corpus.src_vocab), d_embedding)  # 词嵌入层
        self.pos_emb = nn.Embedding.from_pretrained(get_sin_enc_table(corpus.src_len+1, d_embedding), freeze=True) # 位置嵌入层, src_len+1是因为有一个<PAD>token
        self.lawyers = nn.ModuleList(EncoderLayer() for _ in range(n_layers)) # 编码器层列表
    def forward(self, enc_inputs):
        #----------------------维度信息------------------------
        # enc_inputs [batch_size, source_len]
        #-------------------------------------------------------
        # 创建一个从1到source_len的位置索引序列
        pos_indices = torch.arange(1, enc_inputs.size(1) + 1).unsqueeze(0).to(enc_inputs.device)
        #----------------------维度信息------------------------
        # pos_indices [1, source_len]
        #-------------------------------------------------------
        # 将输入序列的索引和位置信息相加，得到位置索引
        enc_outputs = self.src_emb(enc_inputs) + self.pos_emb(pos_indices)
        #----------------------维度信息------------------------
        # enc_outputs [batch_size, source_len, embedding_dim]
        #-------------------------------------------------------
        # 生成自注意力掩码
        enc_self_attn_mask = get_attn_pad_mask(enc_inputs, enc_inputs)
        #----------------------维度信息------------------------
        # enc_self_attn_mask [batch_size, source_len, source_len]
        #-------------------------------------------------------
        enc_self_attn_weights = []  # 用于存储每个编码器层的注意力权重
        # 通过每个编码器层
        for layer in self.lawyers:
            enc_outputs, enc_self_attn_weight = layer(enc_outputs, enc_self_attn_mask)
            enc_self_attn_weights.append(enc_self_attn_weight)
        #----------------------维度信息------------------------
        # enc_outputs [batch_size, seq_len, embedding_dim]与enc_inputs维度相同
        # enc_self_attn_weights [n_layers, batch_size, n_heads, seq_len, seq_len]
        #-------------------------------------------------------
        return enc_outputs, enc_self_attn_weights  # 返回编码器的输出和注意力权重

In [49]:
# 生成后续注意力掩码的函数，用于在多头自注意力计算中忽略未来信息
def get_attn_subsequence_mask(seq):
    #----------------------维度信息------------------------
    # seq的维度是[batch_size, seq_len(Q)=seq_len(K)]
    #-------------------------------------------------------
    # 获取输入序列的形状
    attn_shape = [seq.size(0), seq.size(1), seq.size(1)]
    #----------------------维度信息------------------------
    # attn_shape [batch_size, seq_len, seq_len]
    #-------------------------------------------------------
    # 生成一个下三角矩阵
    subsequent_mask = np.triu(np.ones(attn_shape), k=1)
    #----------------------维度信息------------------------
    # subsequent_mask [batch_size, seq_len, seq_len]
    #-------------------------------------------------------
    # 将numpy数组转换为Tensor，并将其转换为布尔类型
    subsequent_mask = torch.from_numpy(subsequent_mask).bool()
    #----------------------维度信息------------------------
    # subsequent_mask [batch_size, seq_len, seq_len]
    #-------------------------------------------------------
    return subsequent_mask  # 返回后续注意力掩码张量

In [50]:
# 定义解码器层类
class DecoderLayer(nn.Module):
    def __init__(self):
        super(DecoderLayer, self).__init__()
        self.dec_self_attn = MultiHeadAttention()  # 解码器自注意力层
        self.enc_dec_attn = MultiHeadAttention()  # 编码器-解码器注意力层
        self.pos_ffn = PoswiseFeedForwardNet()  # 前馈神经网络层
    def forward(self, dec_inputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask):
        #----------------------维度信息------------------------
        # dec_inputs [batch_size, target_len, embedding_dim]
        # enc_outputs [batch_size, source_len, embedding_dim]
        # dec_self_attn_mask [batch_size, target_len, target_len]
        # dec_enc_attn_mask [batch_size, target_len, source_len]
        #-------------------------------------------------------
        # 将相同的Q，K，V输入到解码器自注意力层
        dec_outputs, dec_self_attn = self.dec_self_attn(dec_inputs, dec_inputs, dec_inputs, dec_self_attn_mask)
        #----------------------维度信息------------------------
        # dec_outputs [batch_size, target_len, embedding_dim]
        # dec_self_attn [batch_size, n_heads, target_len, target_len]
        #-------------------------------------------------------
        # 将解码器的输出和编码器的输出输入到编码器-解码器注意力层
        dec_outputs, dec_enc_attn = self.enc_dec_attn(dec_outputs, enc_outputs, enc_outputs, dec_enc_attn_mask)
        #----------------------维度信息------------------------
        # dec_outputs [batch_size, target_len, embedding_dim]
        # dec_enc_attn [batch_size, n_heads, target_len, source_len]
        #-------------------------------------------------------    
        # 将编码器-解码器注意力层的输出输入到前馈神经网络层
        dec_outputs = self.pos_ffn(dec_outputs)
        #----------------------维度信息------------------------
        # dec_outputs [batch_size, target_len, embedding_dim]
        #-------------------------------------------------------
        # 返回前馈神经网络层的输出、解码器自注意力权重和编码器-解码器注意力权重
        return dec_outputs, dec_self_attn, dec_enc_attn

In [51]:
# 定义解码器类
n_layers = 6 # 解码器层数
class Decoder(nn.Module):
    def __init__(self, corpus):
        super(Decoder, self).__init__()
        self.tgt_emb = nn.Embedding(len(corpus.tgt_vocab), d_embedding)  # 词嵌入层
        self.pos_emb = nn.Embedding.from_pretrained(get_sin_enc_table(corpus.tgt_len+1, d_embedding), freeze=True) # 位置嵌入层, tgt_len+1是因为有一个<PAD>token
        self.lawyers = nn.ModuleList([DecoderLayer() for _ in range(n_layers)]) # 解码器层列表
    def forward(self, dec_inputs, enc_inputs, enc_outputs):
        #----------------------维度信息------------------------
        # dec_inputs [batch_size, target_len]
        # enc_inputs [batch_size, source_len]
        # enc_outputs [batch_size, source_len, embedding_dim]
        #-------------------------------------------------------
        # 创建一个从1到source_len的位置索引序列
        pos_indices = torch.arange(1, dec_inputs.size(1) + 1).unsqueeze(0).to(dec_inputs)
        #----------------------维度信息------------------------
        # pos_indices [1, target_len]
        #-------------------------------------------------------
        # 将输入序列的索引和位置信息相加，得到位置索引
        dec_outputs = self.tgt_emb(dec_inputs) + self.pos_emb(pos_indices)
        #----------------------维度信息------------------------
        # dec_outputs [batch_size, target_len, embedding_dim]
        #-------------------------------------------------------
        # 生成解码器自注意力掩码和解码器-编码器注意力掩码
        dec_self_attn_pad_mask = get_attn_pad_mask(dec_inputs, dec_inputs)  # 填充掩码
        dec_self_attn_subsequence_mask = get_attn_subsequence_mask(dec_inputs)  # 后续掩码
        dec_self_attn_mask = dec_self_attn_pad_mask.bool() | dec_self_attn_subsequence_mask.bool()
        # dec_self_attn_mask = torch.gt((dec_self_attn_pad_mask + dec_self_attn_subsequence_mask), 0)
        dec_enc_attn_mask = get_attn_pad_mask(dec_inputs, enc_inputs)  # 解码器-编码器掩码
        #----------------------维度信息------------------------
        # dec_self_attn_mask [batch_size, target_len, target_len]
        # dec_enc_attn_mask [batch_size, target_len, source_len]
        #-------------------------------------------------------
        dec_self_attn_weights, dec_enc_attn_weights = [], []  # 用于存储每个解码器层的注意力权重
        # 通过每个解码器层
        for layer in self.lawyers:
            dec_outputs, dec_self_attn, dec_enc_attn = layer(dec_outputs, enc_outputs, dec_self_attn_mask, dec_enc_attn_mask)
            dec_self_attn_weights.append(dec_self_attn)
            dec_enc_attn_weights.append(dec_enc_attn)
        #----------------------维度信息------------------------
        # dec_outputs [batch_size, target_len, embedding_dim]与dec_inputs维度相同
        # dec_self_attn_weights [n_layers, batch_size, n_heads, target_len, target_len]
        # dec_enc_attn_weights [n_layers, batch_size, n_heads, target_len, source_len]
        #-------------------------------------------------------
        # 返回解码器的输出、解码器自注意力权重和编码器-解码器注意力权重
        return dec_outputs, dec_self_attn_weights, dec_enc_attn_weights

In [52]:
# 定义Transformer类
class Transformer(nn.Module):
    def __init__(self, corpus):
        super(Transformer, self).__init__()
        self.encoder = Encoder(corpus)  # 初始化编码器实例
        self.decoder = Decoder(corpus)  # 初始化解码器实例
        # 定义线性投影层，将解码器的输出映射到词表维度
        self.projection = nn.Linear(d_embedding, len(corpus.tgt_vocab), bias=False)
    def forward(self, enc_inputs, dec_inputs):
        #----------------------维度信息------------------------
        # enc_inputs [batch_size, source_len]
        # dec_inputs [batch_size, target_len]
        #-------------------------------------------------------
        # 将输入传递给编码器，并获取编码器输出和自注意力权重
        enc_outputs, enc_self_attns = self.encoder(enc_inputs)
        #----------------------维度信息------------------------
        # enc_outputs [batch_size, source_len, embedding_dim]
        # enc_self_attns [n_layers, batch_size, n_heads, source_len, source_len]
        #-------------------------------------------------------
        # 将编码器的输出和解码器的输入传递给解码器，并获取解码器输出和自注意力权重、编码器-解码器注意力权重
        dec_outputs, dec_self_attns, dec_enc_attns = self.decoder(dec_inputs, enc_inputs, enc_outputs)
        #----------------------维度信息------------------------
        # dec_outputs [batch_size, target_len, embedding_dim]
        # dec_self_attns [n_layers, batch_size, n_heads, target_len, source_len]
        # dec_enc_attns [n_layers, batch_size, n_heads, target_len, source_len]
        #-------------------------------------------------------
        # 将解码器的输出映射到词表维度
        dec_logits = self.projection(dec_outputs)
        #----------------------维度信息------------------------
        # dec_logits [batch_size, target_len, tgt_vocab_size]
        #-------------------------------------------------------
        # 返回预测值，编码器自注意力权重，解码器自注意力权重和编码器-解码器注意力权重
        return dec_logits, enc_self_attns, dec_self_attns, dec_enc_attns

In [53]:
# 数据准备
sentences = [
    ['咖哥 喜欢 小冰', 'KaGe likes XiaoBing'],
    ['我 爱 学习 人工智能', 'I love studying AI'],
    ['深度学习 改变 世界', 'DL changed the world'],
    ['自然 语言 处理 很 强大', 'NLP is so powerful'],
    ['神经网络 非常 复杂', 'Neural-Nets are complex'],
]

In [54]:
from collections import Counter # 导入 Counter 类
# 定义 TranslationCorpus 类
class TranslationCorpus:
    def __init__(self, sentences):
        self.sentences = sentences
        # 计算源语言和目标语言的最大句子长度，并分别加 1 和 2 以容纳填充符和特殊符号
        self.src_len = max(len(sentence[0].split()) for sentence in sentences) + 1
        self.tgt_len = max(len(sentence[1].split()) for sentence in sentences) + 2
        # 创建源语言和目标语言的词汇表
        self.src_vocab, self.tgt_vocab = self.create_vocabularies()
        # 创建索引到单词的映射
        self.src_idx2word = {v: k for k, v in self.src_vocab.items()}
        self.tgt_idx2word = {v: k for k, v in self.tgt_vocab.items()}
    # 定义创建词汇表的函数
    def create_vocabularies(self):
        # 统计源语言和目标语言的单词频率
        src_counter = Counter(word for sentence in self.sentences for word in sentence[0].split())
        tgt_counter = Counter(word for sentence in self.sentences for word in sentence[1].split())        
        # 创建源语言和目标语言的词汇表，并为每个单词分配一个唯一的索引
        src_vocab = {'<pad>': 0, **{word: i+1 for i, word in enumerate(src_counter)}}
        tgt_vocab = {'<pad>': 0, '<sos>': 1, '<eos>': 2, 
                     **{word: i+3 for i, word in enumerate(tgt_counter)}}        
        return src_vocab, tgt_vocab
    # 定义创建批次数据的函数
    def make_batch(self, batch_size, test_batch=False):
        input_batch, output_batch, target_batch = [], [], []
        # 随机选择句子索引
        sentence_indices = torch.randperm(len(self.sentences))[:batch_size]
        for index in sentence_indices:
            src_sentence, tgt_sentence = self.sentences[index]
            # 将源语言和目标语言的句子转换为索引序列
            src_seq = [self.src_vocab[word] for word in src_sentence.split()]
            tgt_seq = [self.tgt_vocab['<sos>']] + [self.tgt_vocab[word] \
                         for word in tgt_sentence.split()] + [self.tgt_vocab['<eos>']]            
            # 对源语言和目标语言的序列进行填充
            src_seq += [self.src_vocab['<pad>']] * (self.src_len - len(src_seq))
            tgt_seq += [self.tgt_vocab['<pad>']] * (self.tgt_len - len(tgt_seq))            
            # 将处理好的序列添加到批次中
            input_batch.append(src_seq)
            output_batch.append([self.tgt_vocab['<sos>']] + ([self.tgt_vocab['<pad>']] * \
                                    (self.tgt_len - 2)) if test_batch else tgt_seq[:-1])
            target_batch.append(tgt_seq[1:])        
          # 将批次转换为 LongTensor 类型
        input_batch = torch.LongTensor(input_batch)
        output_batch = torch.LongTensor(output_batch)
        target_batch = torch.LongTensor(target_batch)            
        return input_batch, output_batch, target_batch
# 创建语料库类实例
corpus = TranslationCorpus(sentences)

In [55]:
# 训练Transformer模型
import torch
import torch.optim as optim
model = Transformer(corpus)  # 创建Transformer实例
criterion = nn.CrossEntropyLoss()  # 定义交叉熵损失函数\
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # 定义Adam优化器
epochs = 100
for epoch in range(epochs):
    optimizer.zero_grad()
    enc_inputs, dec_inputs, tgt_batch = corpus.make_batch(batch_size)  # 创建批次数据
    outputs, _, _, _ = model(enc_inputs, dec_inputs)  # 前向传播
    loss = criterion(outputs.view(-1, len(corpus.tgt_vocab)), tgt_batch.view(-1))  # 计算损失
    if (epoch + 1) % 20 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()  # 反向传播
    optimizer.step()  # 更新参数

Epoch: 0020 cost = 0.036936
Epoch: 0040 cost = 0.004030
Epoch: 0060 cost = 0.001788
Epoch: 0080 cost = 0.001256
Epoch: 0100 cost = 0.001012


In [56]:
# 测试Transformer模型
# 创建一个大小为1的批次，目标语言序列在测试阶段，仅包含句子开始符号<sos>
enc_inputs, dec_inputs, tgt_batch = corpus.make_batch(batch_size=1, test_batch=True)  # 创建批次数据
predict, enc_self_attns, dec_self_attns, dec_enc_attns = model(enc_inputs, dec_inputs)  # 使用模型进行翻译
predict = predict.view(-1, len(corpus.tgt_vocab))  # 获取预测值
predict = predict.data.max(1, keepdim=True)[1]  # 获取预测值的索引
# 打印源语言和目标语言的句子
print('Source:', ' '.join([corpus.src_idx2word[src] for src in enc_inputs.data.numpy()[0]]))
print('Target:', ' '.join([corpus.tgt_idx2word[tgt] for tgt in tgt_batch.data.numpy()[0]]))

Source: 咖哥 喜欢 小冰 <pad> <pad> <pad>
Target: KaGe likes XiaoBing <eos> <pad>
